# Data Cleaning & Preprocessing

## Objective
Based on the issues identified during Day 1 data profiling, this phase focuses on improving dataset quality by cleaning inconsistencies, handling placeholder values, treating outliers, and preparing variables for analysis.

## Issues Identified in Day 1
- Presence of "unknown" values in categorical columns
- Potential outliers in numeric columns
- Columns with very high unknown proportions
- Need for feature preparation before analysis

This notebook performs systematic preprocessing steps to resolve these issues.

In [1]:
import pandas as pd
df = pd.read_csv("day1_output.csv")

In [2]:
#Fix Data Types
df["age"] = df["age"].astype(int)
df["balance"] = df["balance"].astype(int)

df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

**All numerical and categorical columns were checked and corrected to ensure accurate analysis. Proper datatypes prevent calculation errors and improve performance.**

In [3]:
#Handling Missing values

for col in df.select_dtypes(include="object"):
    print(col, df[col].unique())

    df["job"].replace("unknown", pd.NA, inplace=True)
df["education"].replace("unknown", pd.NA, inplace=True)
df["contact"].replace("unknown", pd.NA, inplace=True)
df["poutcome"].replace("unknown", pd.NA, inplace=True)

job ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital ['married' 'single' 'divorced']
education ['tertiary' 'secondary' 'unknown' 'primary']
default ['no' 'yes']
housing ['yes' 'no']
loan ['no' 'yes']
contact ['unknown' 'cellular' 'telephone']
month ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome ['unknown' 'failure' 'other' 'success']
y ['no' 'yes']


**Categorical variables were standardized by converting text to lowercase and removing extra spaces to ensure consistency across categories.**

In [4]:
#Remove Duplicates
df.duplicated().sum()
df = df.drop_duplicates()

**Duplicate records were checked and removed to prevent bias in analysis.**

In [5]:
#Handle Missing Values
df = df.replace("unknown", pd.NA)

df["job"].fillna(df["job"].mode()[0], inplace=True)
df["education"].fillna(df["education"].mode()[0], inplace=True)

df["balance"] = df["balance"].fillna(df["balance"].median())

df["contact"].fillna("not_specified", inplace=True)
df.drop("poutcome", axis=1, inplace=True)

# fix age_group
df["age_group"] = pd.cut(
    df["age"],
    bins=[0,30,45,60,120],
    labels=["Young","Adult","Middle Age","Senior"],
    include_lowest=True
)

df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
y            0
age_group    0
dtype: int64

Placeholder values such as "unknown" were treated as missing values and replaced using mode imputation for categorical columns. This preserves dataset size while maintaining realistic distributions.

In [6]:
# Standardize text formatting across all categorical columns
for col in df.select_dtypes(include="object"):
    df[col] = df[col].apply(lambda x: x.title() if isinstance(x, str) else x)

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,y,age_group
0,58,Management,Married,Tertiary,No,2143,Yes,No,Not_Specified,5,May,261,1,-1,0,No,Middle Age
1,44,Technician,Single,Secondary,No,29,Yes,No,Not_Specified,5,May,151,1,-1,0,No,Adult
2,33,Entrepreneur,Married,Secondary,No,2,Yes,Yes,Not_Specified,5,May,76,1,-1,0,No,Adult
3,47,Blue-Collar,Married,Secondary,No,1506,Yes,No,Not_Specified,5,May,92,1,-1,0,No,Middle Age
4,33,Blue-Collar,Single,Secondary,No,1,No,No,Not_Specified,5,May,198,1,-1,0,No,Adult


In [7]:
#Outlier Detection (IQR Method)

def detect_outliers(col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    return df[(df[col] < lower) | (df[col] > upper)]

detect_outliers("balance")

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,y,age_group
34,51,Management,Married,Tertiary,No,10635,Yes,No,Not_Specified,5,May,336,1,-1,0,No,Middle Age
65,51,Management,Married,Tertiary,No,6530,Yes,No,Not_Specified,5,May,91,1,-1,0,No,Middle Age
69,35,Blue-Collar,Single,Secondary,No,12223,Yes,Yes,Not_Specified,5,May,177,1,-1,0,No,Adult
70,57,Blue-Collar,Married,Secondary,No,5935,Yes,Yes,Not_Specified,5,May,258,1,-1,0,No,Middle Age
186,40,Services,Divorced,Secondary,No,4384,Yes,No,Not_Specified,5,May,315,1,-1,0,No,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45164,35,Services,Married,Tertiary,No,4655,No,No,Cellular,9,Nov,111,2,-1,0,No,Adult
45181,46,Blue-Collar,Married,Secondary,No,6879,No,No,Cellular,15,Nov,74,2,118,3,No,Middle Age
45185,60,Services,Married,Tertiary,No,4256,Yes,No,Cellular,16,Nov,200,1,92,4,Yes,Middle Age
45191,75,Retired,Divorced,Tertiary,No,3810,Yes,No,Cellular,16,Nov,262,1,183,1,Yes,Senior


Outliers were detected using the IQR method. Extreme values outside the acceptable range were removed to prevent distortion in statistical analysis and modeling.

In [8]:
#Transformations

import numpy as np

# clean balance first
df["balance"] = df["balance"].replace([np.inf, -np.inf], np.nan)
df["balance"] = df["balance"].fillna(df["balance"].median())

# shift values so all positive
df["balance"] = df["balance"] - df["balance"].min() + 1

# log transform
df["balance"] = np.log1p(df["balance"])

# check skewness
df["balance"].skew()

np.float64(1.4669168081279749)

Log transformation was applied to skewed numeric variables to normalize their distribution and improve statistical reliability.

In [9]:
import os
os.makedirs("data/interim", exist_ok=True)

In [10]:
df.to_csv("data/interim/cleaned_day2.csv", index=False)

## Day 2 Summary

Data cleaning and preprocessing were successfully completed. Key improvements include:
- standardized categorical values
- handled placeholder missing values
- removed duplicates
- treated outliers
- transformed skewed variables

The dataset is now clean, consistent, and ready for exploratory analysis.